In [1]:
from src.hypotheses import generate_probability_matrix_2D, update_hyps_scores, get_last_detection_id

In [2]:
import pandas as pd
import numpy as np

In [3]:
detections = pd.read_csv('../figs/detections.csv')
faunastep = 0.1
bg_prob = 10/25
detections.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

prbz = generate_probability_matrix_2D(detections, faunastep)

/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:74: RuntimeWarning: invalid value encountered in true_divide
  z_stat = np.divide(dists, faunastep * Dtime)


In [4]:
scores = []
hyps = []
t0 = detections.time.unique()[0]

In [5]:
for i, row in detections[detections["time"] == t0].iterrows():
    hyps.append([[row["id"]]])
    scores.append(bg_prob)


In [6]:
detections

,id,time,individual,x,y,camera_x,camera_y
0,0,11,1.0,3.701638,0.382557,3.300000e+00,0.0
1,1,12,1.0,3.600209,0.327829,3.600000e+00,0.0
2,2,13,1.0,3.652598,0.236747,3.900000e+00,0.0
3,3,14,1.0,3.706451,0.233060,4.200000e+00,0.0
4,4,24,2.0,3.168524,0.331691,3.600000e+00,0.6
5,5,25,2.0,3.168922,0.114513,3.300000e+00,0.6
6,6,26,2.0,3.203036,0.181815,3.000000e+00,0.6
7,7,43,0.0,1.346057,1.380389,9.000000e-01,1.2
8,8,44,0.0,1.257174,1.506333,1.200000e+00,1.2
9,9,45,0.0,1.176147,1.512717,1.500000e+00,1.2


In [35]:
def update_hyps_scores_gate(det, hyps, scores, probz, bg_prob, score_thresh):
    new_hyps = []
    new_scores = []

    detection_id = det.iloc[0].id
    #     detection_time = det.iloc[0].time
    #     detection_pos = det.iloc[0].object_location

    # go through hypotheses
    for hyp_i, hyp in enumerate(hyps):
        existing_score = scores[hyp_i]

        # scenario where the new detection is a new individual
        # pad existing branches

        new_scen = [h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0])) * np.nan) + [detection_id]]
        new_hyps.append(new_scen)

        # score is the existing + background prob
        new_scores.append(existing_score + bg_prob)

        # go through the branches in the hypothesis
        for i, branch in enumerate(hyp):
            # calculate score
            last_detection = get_last_detection_id(branch)
            scre = probz[int(last_detection), int(detection_id)]
            
            # Do GATING, check if score is above threshold
            if scre > score_thresh:
                # scenario where the new detection is a repeat
                repeat_scen = [branch + [detection_id]]
                repeat_scen += [hyp[j] + [np.nan] for j in range(len(hyp)) if j != i]

                new_hyps.append(repeat_scen)

                new_scores.append(existing_score + scre)
            else:
                print("Gated repeat detection {}, {}".format(last_detection, detection_id))
                continue
                
    return new_hyps, new_scores

In [36]:
prbz

array([[       nan, 0.24910963, 0.44178921, ..., 0.76538035, 0.7653527 ,
        0.76634531],
       [0.24910963,        nan, 0.29337975, ..., 0.76257865, 0.76272098,
        0.76388891],
       [0.44178921, 0.29337975,        nan, ..., 0.75608388, 0.75623234,
        0.75741376],
       ...,
       [0.76538035, 0.76257865, 0.75608388, ...,        nan, 0.18371355,
        0.1881364 ],
       [0.7653527 , 0.76272098, 0.75623234, ..., 0.18371355,        nan,
        0.18868952],
       [0.76634531, 0.76388891, 0.75741376, ..., 0.1881364 , 0.18868952,
               nan]])

In [39]:
det = detections[detections.time == detections.time.unique()[1]]
nh_gated, ns_gated = update_hyps_scores_gate(det, hyps, scores, prbz, bg_prob, 0.5)

Gated repeat detection 0.0, 1.0


In [41]:
nh_ungated, ns_ungated = update_hyps_scores(det, hyps, scores, prbz, bg_prob)

In [42]:
nh_ungated

[[[0.0, nan], [nan, 1.0]], [[0.0, 1.0]]]